# E-mail SLA Calculator for DMO Material Requests

In [1]:
import win32com.client as w32
import pandas
from numpy import timedelta64 as td
pandas.set_option('mode.chained_assignment', None)

### Connect to Outlook and extract the [Sent Items] folder in a variable

In [2]:
sent_items = w32.Dispatch('Outlook.Application').GetNameSpace('MAPI').Folders('DMO Material Requests').Folders('Sent Items')

 ### Save all the e-mails in the sent_items in a variable

In [3]:
sent_mails = sent_items.Items

### Save all the employees in a dictionary

In [4]:
temp_dmo = {'Catalin Paltanea': 'Catalin Paltanea',
 'Irina Jiga': 'Irina Jiga',
 'Claudiu Catargiu': 'Claudiu Catargiu',
 'Cezara Stefania Neagu': 'Cezara Stefania Neagu',
 'Arabella Stan': 'Arabella Stan',
 'Valentin Vulpe': 'Valentin Vulpe',
 'Bogdan Dragan': 'Bogdan Dragan',
 'Razvan Rusescu': 'Razvan Rusescu',
 'Razvan-Alexandru Florian': 'Razvan-Alexandru Florian',
 'Rontu Edmond': 'Rontu Edmond',
 'Secu Gabriel': 'Secu Gabriel',
 'Marius Savu': 'Marius Savu'}

### Create blank lists in which all the e-mail data will be stored

In [5]:
sender, subject, so, generic_sender, convoID = [],[],[],[],[]

### Loop through all e-mails and extract the sender, subject, sent on date & time, the person from the team who sent the e-mail from the generic mailbox and the conversation id

In [6]:
for mail in sent_mails:
    sender.append(mail.SenderName)
    subject.append(mail.Subject)
    so.append(str(mail.SentOn))
    if '26ade4a6050840e798d653178cb0864c-DMO Materia'.upper() in mail.SenderEmailAddress.upper():
        contor = 0
        for element in temp_dmo:
            if element in mail.Body:
                generic_sender.append('{0}, {1}'.format(temp_dmo[element].split()[1],temp_dmo[element].split()[0]))
                break
            else:
                contor += 1
                if contor == len(temp_dmo):
                    generic_sender.append('No sender found')
                    break
                else:
                    continue
    else:
        generic_sender.append(None)
    convoID.append(mail.ConversationID)

### Let's check if all the lists have the same number of details so we can transpose them to a dataframe

In [7]:
print('''
Senders: {}
Subjects: {}
SentOns: {}
Generic Senders: {}
Conversation IDs: {}
'''.format(len(sender),len(subject),len(so),len(generic_sender),len(convoID)))


Senders: 7538
Subjects: 7538
SentOns: 7538
Generic Senders: 7538
Conversation IDs: 7538



### If all lists have the same size we can transpose them into a pandas dataframe

In [8]:
df = pandas.DataFrame({'Sender': sender, 'Subject': subject,
                                    'ConversationID': convoID, 'SentOn': so,
                                    'MailBox': 'Sent Items', 'DMO_sender': generic_sender})

### Let's have a quick look at what the dataframe looks like

In [9]:
df

,Sender,Subject,ConversationID,SentOn,MailBox,DMO_sender
0,DMO Material Requests,FW: Crown Effective Out Date Updates,F8147819100A4B548EE30447FEE25DAF,2020-03-12 11:44:41.985000+00:00,Sent Items,No sender found
1,DMO Material Requests,RE: BOM ASSIGNMENT - Soft Change Cleanup,6F3BBB79512B40558CFE65F5C0B3F36A,2020-03-12 11:38:13.861000+00:00,Sent Items,No sender found
2,DMO Material Requests,RE: Please adjut 10010384 quantity,AD02799C9E5B4FEA88FA5DC7A08415B6,2020-03-11 21:46:10.437000+00:00,Sent Items,No sender found
3,DMO Material Requests,RE: [EXTERNAL] New Materials,EE469A878EE442CCADE63CDD4CA0198E,2020-03-11 21:03:52.917000+00:00,Sent Items,No sender found
4,DMO Material Requests,Project 7337 - Cape Line to FTW (Project Domin...,088AE2733C95455A97D3E159CA49AD13,2020-03-11 20:48:30.239000+00:00,Sent Items,No sender found
...,...,...,...,...,...,...
7533,DMO Material Requests,RE: MRP Value Request - Lot Size,3ED7FFDB8BEC452E9E83011E438C76FC,2020-05-06 12:16:55+00:00,Sent Items,No sender found
7534,DMO Material Requests,RE: MRP Value Request - Lot Size,3ED7FFDB8BEC452E9E83011E438C76FC,2020-05-06 12:16:50.316000+00:00,Sent Items,No sender found
7535,DMO Material Requests,RE: URGENT 4.7.2020 BOM Assignment START 7426 ...,EC99297363454502BDB69C5C20B1212D,2020-05-06 12:24:03+00:00,Sent Items,No sender found
7536,DMO Material Requests,RE: URGENT 4.7.2020 BOM Assignment START 7426 ...,EC99297363454502BDB69C5C20B1212D,2020-05-06 12:23:59.308000+00:00,Sent Items,No sender found


### Let's export the dataframe to excel for now just to make sure we don't have to extract the data all over again in case something is wrong

In [10]:
df.to_excel('Raw Data E-mail SLA Calculator - Sent Items DMO.xlsx')

### Let's edit the data frame by deleting the miliseconds from the 'Sent On' column, and let's sort the dataframe based on 'Conversation ID' and 'Sent On'

In [11]:
df['SentOn'] = df['SentOn'].str.strip('+00:00')
df['SentOn'] = pandas.to_datetime(df['SentOn'])
df = df.sort_values(['ConversationID', 'SentOn'], ascending=(False, True))
df.reset_index(inplace=True)
df.drop(columns='index', inplace=True)

### Let's create a separate dataframe only with the unique conversation ID's so we can assess the number of tickets handled by the team

In [12]:
df2 = pandas.DataFrame()
dcid = pandas.DataFrame({'DCID': []})
dcid['DCID'] = df['ConversationID'].unique()

### Loop through each distinct conversation id, create a temporary dataframe where the e-mail response time is calculated and return the response time in the original dataframe

In [13]:
for i in dcid.iterrows():
    tempdf = df[df['ConversationID'] == i[1]['DCID']]
    tempdf['Response_Time'] = tempdf['SentOn'].diff()
    df2 = df2.append(tempdf)
df2 = df2['Response_Time']
df = df.merge(df2.to_frame(), how='left', left_index=True, right_index=True)
df['Response_Time_h'] = df['Response_Time'] / td(1, 'h')
df['Response_Time_days'] = df['Response_Time'] / td(1, 'D')
df.drop(columns=['Response_Time'], inplace=True)
df['SentOn_day'] = df['SentOn'].dt.day_name()
df['SentOn_Month'] = df['SentOn'].dt.month_name()
df['SentOn_Year'] = df['SentOn'].dt.year.astype(str)

### Create a separate dataframe which will calculate the total response time for each ticket

In [14]:
total_response_time = df.groupby(['ConversationID'], as_index=False)[['Response_Time_h',
                                                                                'Response_Time_days']].sum()
total_response_time.rename(columns={'Response_Time_h': 'Total_Response_Time_h',
                                    'Response_Time_days': 'Total_Response_Time_days'}, inplace=True)
unique_conversationIDs = df['ConversationID'].drop_duplicates().to_frame()

### Now let's export all three dataframes to an excel file

In [15]:
writer = pandas.ExcelWriter('E-mail SLA calculator Sent Items.xlsx',engine='xlsxwriter')

with writer:
    df.to_excel(writer, index=False, sheet_name='Raw Data')
    total_response_time.to_excel(writer, index=False, sheet_name='Response Time summary')
    unique_conversationIDs.to_excel(writer, index=False, sheet_name='Unique Conversation IDs')
writer.save()